In [2]:
from _lumber_prices import *
from _lumber_schema import *
from adjusted_future import *

import pandas as pd  
import numpy as np
from datetime import datetime
from os import listdir
from os.path import isfile, join
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen

In [ ]:
Lum_3 = get_ctrct_mnth_df(ctrct_sequence=3)


In [10]:
Lum_2

,high,low,close,volume,openInterest,symbol,expirationDate,Days_to_expiry,cash_price
2022-08-10,603.000000,571.200012,603.000000,NaN,NaN,LBX22,2022-11-15,70.0,580.0
2022-08-09,570.700012,540.000000,554.000000,145.0,714.0,LBX22,2022-11-15,71.0,580.0
2022-08-08,539.200012,496.000000,530.000000,111.0,717.0,LBX22,2022-11-15,72.0,590.0
2022-08-05,518.500000,490.200012,490.200012,68.0,678.0,LBX22,2022-11-15,73.0,590.0
2022-08-04,507.700012,480.000000,504.200012,121.0,670.0,LBX22,2022-11-15,74.0,590.0
...,...,...,...,...,...,...,...,...,...
1969-10-07,82.000000,81.500000,82.000000,NaN,NaN,LBK70,1970-05-15,159.0,NaN
1969-10-06,81.500000,81.500000,81.500000,NaN,NaN,LBK70,1970-05-15,160.0,NaN
1969-10-03,81.500000,81.500000,81.500000,NaN,NaN,LBK70,1970-05-15,161.0,NaN
1969-10-02,81.250000,81.000000,81.250000,NaN,NaN,LBK70,1970-05-15,162.0,NaN


In [9]:
Lum_3

,high,low,close,volume,openInterest,symbol,expirationDate,Days_to_expiry,cash_price
2022-08-10,636.000000,620.000000,636.000000,NaN,NaN,LBF23,2023-01-13,113.0,580.0
2022-08-09,600.299988,587.599976,587.599976,33.0,52.0,LBF23,2023-01-13,114.0,580.0
2022-08-08,564.000000,542.099976,561.000000,12.0,76.0,LBF23,2023-01-13,115.0,590.0
2022-08-05,526.000000,522.500000,522.500000,6.0,66.0,LBF23,2023-01-13,116.0,590.0
2022-08-04,517.900024,509.000000,521.000000,13.0,62.0,LBF23,2023-01-13,117.0,590.0
...,...,...,...,...,...,...,...,...,...
1969-10-07,83.000000,82.500000,83.000000,NaN,NaN,LBN70,1970-07-15,202.0,NaN
1969-10-06,83.000000,83.000000,83.000000,NaN,NaN,LBN70,1970-07-15,203.0,NaN
1969-10-03,83.000000,83.000000,83.000000,NaN,NaN,LBN70,1970-07-15,204.0,NaN
1969-10-02,84.000000,84.000000,84.000000,NaN,NaN,LBN70,1970-07-15,205.0,NaN


In [3]:
# Takes 2 minutes to run
Lum_1 = get_ctrct_mnth_df(ctrct_sequence=1)
Lum_2 = get_ctrct_mnth_df(ctrct_sequence=2)
Lum_3 = get_ctrct_mnth_df(ctrct_sequence=3)

Lum_1.head(2).append(Lum_1.tail(2))

,high,low,close,volume,openInterest,symbol,expirationDate,Days_to_expiry,cash_price
2022-08-10,601.799988,573.000000,601.799988,NaN,NaN,LBU22,2022-09-15,27.0,580.0
2022-08-09,565.000000,528.400024,552.799988,376.0,1895.0,LBU22,2022-09-15,28.0,580.0
1969-10-02,80.500000,79.500000,80.500000,NaN,NaN,LBH70,1970-03-13,117.0,NaN
1969-10-01,80.250000,78.000000,80.000000,NaN,NaN,LBH70,1970-03-13,118.0,NaN


In [45]:
def calculate_adj_ret(Lum_1,Lum_2, switch_list, switch_second):
    Adj_Lum = Lum_1.copy(deep=True)
    cols_to_replace = ['symbol', 'expirationDate', 'Days_to_expiry','close','cash_price']
    Adj_Lum = Lum_1[cols_to_replace]
    Adj_Lum['Switch'] = switch_list
    Adj_Lum['Next_Contract_Close'] = np.nan

    # Fill the Next_Contract_Close with the second contract close *
    for dt in Adj_Lum.index: 
        if (dt in Lum_1.index) & (dt in Lum_2.index):
            Adj_Lum.loc[dt,'Next_Contract_Close'] = Lum_2.loc[dt,'close']

    # Fill rows with second contract where switch is 1 when switch_second equals True - This column is used to calculate returns for same contract
    if switch_second == True:
        for dt in Adj_Lum.index:
            if (Adj_Lum.loc[dt, 'Switch'] == 1) & (dt in Lum_2.index):
                for col in cols_to_replace:
                    Adj_Lum.loc[dt, col] = Lum_2.loc[dt, col]
                    
                    # * But when switch_second equals True, we need to make the next contract as third
                    if (dt in Lum_2.index) & (dt in Lum_3.index):
                        Adj_Lum.loc[dt, 'Next_Contract_Close'] = Lum_3.loc[dt, 'close']

    # Fill Next Close column (previous date) with 2nd Contract's Close Price (previous date) wherever switch is 1
    for l,dt in zip(range(len(Adj_Lum)),Adj_Lum.index):
        try:
            idx = Adj_Lum.index.get_loc(Adj_Lum[Adj_Lum.Switch == 1].index[l])
            dt_to_replace = Adj_Lum.index[idx+1]
            Adj_Lum.loc[dt_to_replace, 'Next_Close'] = Lum_2.loc[dt_to_replace, 'close']
        except:
            pass

    # Calculate Adj Returns - Where switch is 0, calculate simple log returns based on only 'Close' column but when Switch is 1, calculate log returns based on 'Close' and 'Next_Close' columns
    for l,dt in zip(range(len(Adj_Lum)),Adj_Lum.index):
        try:
            if (Adj_Lum.loc[dt, 'Switch'] == 1):
                Adj_Lum.loc[dt, 'Adj_ret'] = np.log(Adj_Lum.iloc[l]['close']/Adj_Lum.iloc[l+1]['Next_Close'])
            else:
                Adj_Lum.loc[dt, 'Adj_ret'] = np.log(Adj_Lum.iloc[l]['close']/Adj_Lum.iloc[l+1]['close'])
            
        except:
            pass
    
    Adj_Lum['Basis'] = Adj_Lum['cash_price'] - Adj_Lum['close']
    Adj_Lum['BasisByDte'] = Adj_Lum['Basis'] / Adj_Lum['Days_to_expiry']
    Adj_Lum['First-Second'] = Adj_Lum['close'] - Adj_Lum['Next_Contract_Close']
    Adj_Lum.drop(['Switch','Next_Close','Next_Contract_Close'], axis=1, inplace=True)

    return Adj_Lum

In [46]:
# 1. Never Switching - We still need to use previous close price on day of expiry since we need to calculate adj return when new contract starts 
# In this case just shifting list to the left - the last index won't matter as it is
switch_list_1 = [item for sublist in np.where([Lum_1.index == Lum_1.expirationDate],1,0).tolist() for item in sublist]
switch_list_1.append(switch_list_1.pop(0))
# len(switch_list_1)

# 2. Switching on the day of expiry
switch_list_2 = [item for sublist in np.where([Lum_1.index == Lum_1.expirationDate],1,0).tolist() for item in sublist]
# len(switch_list_2)

# 3. Switch to 2nd month at the start of contract expiry month
dt_lists = []
for end_dt in Lum_1.expirationDate.unique():
    end_dt = pd.to_datetime(end_dt)
    for dt in pd.date_range(start=end_dt.strftime("%Y-%m-1"), end=end_dt):
        if dt in Lum_1.index:
            dt_lists.append(dt)

switch_list_3 = [1 if dt in dt_lists else 0 for dt in Lum_1.index]
# len(switch_list_3)

In [47]:
# Takes 1 minute to run
Adj_Lum_1 = calculate_adj_ret(Lum_1,Lum_2, switch_list_1,switch_second=False)
Adj_Lum_2 = calculate_adj_ret(Lum_1,Lum_2, switch_list_2,switch_second=True)
Adj_Lum_3 = calculate_adj_ret(Lum_1,Lum_2, switch_list_3,switch_second=True)

Adj_Lum_1.head(2).append(Adj_Lum_1.tail(2))

,symbol,expirationDate,Days_to_expiry,close,cash_price,Next_Contract_Close,Adj_ret,Basis,BasisByDte,First-Second
2022-08-10,LBU22,2022-09-15,27.0,601.799988,580.0,603.00,0.084929,-21.799988,-0.807407,-1.200012
2022-08-09,LBU22,2022-09-15,28.0,552.799988,580.0,554.00,0.062515,27.200012,0.971429,-1.200012
1969-10-02,LBH70,1970-03-13,117.0,80.500000,NaN,81.25,0.006231,NaN,NaN,-0.750000
1969-10-01,LBH70,1970-03-13,118.0,80.000000,NaN,80.75,NaN,NaN,NaN,-0.750000


In [58]:
Lum_1[17:23]

,high,low,close,volume,openInterest,symbol,expirationDate,Days_to_expiry,cash_price
2022-07-18,NaN,626.200012,640.799988,176.0,1439.0,LBU22,2022-09-15,44.0,670.0
2022-07-15,682.000000,655.000000,682.000000,187.0,1428.0,LBN22,2022-07-15,1.0,670.0
2022-07-14,645.700012,635.000000,640.000000,50.0,2.0,LBN22,2022-07-15,2.0,670.0
2022-07-13,660.000000,643.500000,652.000000,65.0,36.0,LBN22,2022-07-15,3.0,660.0
2022-07-12,656.599976,643.000000,645.000000,33.0,101.0,LBN22,2022-07-15,4.0,660.0
2022-07-11,670.000000,640.299988,661.099976,34.0,118.0,LBN22,2022-07-15,5.0,640.0


In [59]:
Adj_Lum_1[17:23]

,symbol,expirationDate,Days_to_expiry,close,cash_price,Next_Contract_Close,Adj_ret,Basis,BasisByDte,First-Second
2022-07-18,LBU22,2022-09-15,44.0,640.799988,670.0,642.599976,-0.012407,29.200012,0.663637,-1.799988
2022-07-15,LBN22,2022-07-15,1.0,682.000000,670.0,648.799988,0.063561,-12.000000,-12.000000,33.200012
2022-07-14,LBN22,2022-07-15,2.0,640.000000,670.0,646.000000,-0.018576,30.000000,15.000000,-6.000000
2022-07-13,LBN22,2022-07-15,3.0,652.000000,660.0,683.299988,0.010794,8.000000,2.666667,-31.299988
2022-07-12,LBN22,2022-07-15,4.0,645.000000,660.0,679.299988,-0.024655,15.000000,3.750000,-34.299988
2022-07-11,LBN22,2022-07-15,5.0,661.099976,640.0,696.799988,0.003182,-21.099976,-4.219995,-35.700012


In [62]:
Lum_2[17:23]

,high,low,close,volume,openInterest,symbol,expirationDate,Days_to_expiry,cash_price
2022-07-18,647.500000,NaN,642.599976,24.0,373.0,LBX22,2022-11-15,87.0,670.0
2022-07-15,650.000000,618.099976,648.799988,15.0,362.0,LBU22,2022-09-15,45.0,670.0
2022-07-14,664.200012,634.299988,646.000000,277.0,1449.0,LBU22,2022-09-15,46.0,670.0
2022-07-13,695.000000,670.200012,683.299988,121.0,1456.0,LBU22,2022-09-15,47.0,660.0
2022-07-12,700.000000,671.500000,679.299988,125.0,1489.0,LBU22,2022-09-15,48.0,660.0
2022-07-11,707.000000,658.099976,696.799988,199.0,1490.0,LBU22,2022-09-15,49.0,640.0


In [60]:
Adj_Lum_2[17:23]

,symbol,expirationDate,Days_to_expiry,close,cash_price,Next_Contract_Close,Adj_ret,Basis,BasisByDte,First-Second
2022-07-18,LBU22,2022-09-15,44.0,640.799988,670.0,642.599976,-0.012407,29.200012,0.663637,-1.799988
2022-07-15,LBU22,2022-09-15,45.0,648.799988,670.0,640.099976,0.004325,21.200012,0.471111,8.700012
2022-07-14,LBN22,2022-07-15,2.0,640.000000,670.0,646.000000,-0.018576,30.000000,15.000000,-6.000000
2022-07-13,LBN22,2022-07-15,3.0,652.000000,660.0,683.299988,0.010794,8.000000,2.666667,-31.299988
2022-07-12,LBN22,2022-07-15,4.0,645.000000,660.0,679.299988,-0.024655,15.000000,3.750000,-34.299988
2022-07-11,LBN22,2022-07-15,5.0,661.099976,640.0,696.799988,0.003182,-21.099976,-4.219995,-35.700012


In [72]:
Lum_2[27:33]

,high,low,close,volume,openInterest,symbol,expirationDate,Days_to_expiry,cash_price
2022-07-04,NaN,NaN,669.700012,NaN,NaN,LBU22,2022-09-15,54.0,630.0
2022-07-01,686.299988,627.500000,669.700012,NaN,NaN,LBU22,2022-09-15,55.0,630.0
2022-06-30,643.400024,587.500000,637.299988,389.0,1670.0,LBU22,2022-09-15,56.0,630.0
2022-06-29,602.200012,580.200012,594.400024,299.0,1680.0,LBU22,2022-09-15,57.0,620.0
2022-06-28,597.500000,576.799988,581.200012,270.0,1556.0,LBU22,2022-09-15,58.0,620.0
2022-06-27,592.299988,574.000000,583.500000,111.0,1485.0,LBU22,2022-09-15,59.0,600.0


In [69]:
Adj_Lum_3[27:33]

,symbol,expirationDate,Days_to_expiry,close,cash_price,Next_Contract_Close,Adj_ret,Basis,BasisByDte,First-Second
2022-07-04,LBU22,2022-09-15,54.0,669.700012,630.0,655.799988,0.000000,-39.700012,-0.735185,13.900024
2022-07-01,LBU22,2022-09-15,55.0,669.700012,630.0,655.799988,0.049589,-39.700012,-0.721818,13.900024
2022-06-30,LBN22,2022-07-15,12.0,663.299988,630.0,637.299988,0.045810,-33.299988,-2.774999,26.000000
2022-06-29,LBN22,2022-07-15,13.0,633.599976,620.0,594.400024,0.025577,-13.599976,-1.046152,39.199951
2022-06-28,LBN22,2022-07-15,14.0,617.599976,620.0,581.200012,0.010744,2.400024,0.171430,36.399963
2022-06-27,LBN22,2022-07-15,15.0,611.000000,600.0,583.500000,0.003936,-11.000000,-0.733333,27.500000
